In [29]:
%pip install -r requirements.txt
from PIL import Image
import random
import numpy as np
from scipy.ndimage import gaussian_filter
import torch
import torch.nn as nn
import torch.optim as optim

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
def generate_images(image_size=100, output_to_file=False):
    # Create random grayscale image
    features = np.random.randint(0, 256, (image_size, image_size), dtype=np.uint8)

    # Create and save the raw image if needed
    image = Image.fromarray(features)
    if output_to_file:
        image.save('input.png')

    # Apply Gaussian filter
    filtered_image = gaussian_filter(features.astype(np.float32), sigma=0.7)

    # Normalize filtered image to range [0, 255] and convert to uint8
    filtered_image = np.clip(filtered_image, 0, 255).astype(np.uint8)

    # Create and save the filtered image if needed
    image = Image.fromarray(filtered_image)
    if output_to_file:
        image.save('output.png')

    return features.flatten().astype(np.float32), filtered_image.flatten().astype(np.float32)



In [5]:
generate_images()[0].shape

(10000,)

In [11]:
# Store tensors in a list
X_list = []
Y_list = []

for n in range(10000):
    X_n, Y_n = generate_images()
    X_list.append(torch.tensor(X_n, dtype=torch.float32))
    Y_list.append(torch.tensor(Y_n, dtype=torch.float32))

# Stack tensors into a single tensor
X = torch.stack(X_list)
Y = torch.stack(Y_list)

print(X.shape, Y.shape, Y[0].shape)



torch.Size([10000, 10000]) torch.Size([10000, 10000]) torch.Size([10000])


In [32]:
class LinearRegression(nn.Module):
    def __init__(self, input_dim, hidden_dims=[64, 32], output_dim=1, dropout_rate=0.0):
        super(LinearRegression, self).__init__()
        
        layers = []
        prev_dim = input_dim

        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())  # Activation function
            if dropout_rate > 0:
                layers.append(nn.Dropout(dropout_rate))  # Optional dropout
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, output_dim))  # Output layer
        
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x).view(-1)  # Ensure correct shape

def train(model, x, y, num_epochs=100, learning_rate=0.01, shuffle=True, X_test=None, y_test=None, device="cpu"):
    x, y = x.to(device), y.to(device)
    if X_test is not None and y_test is not None:
        X_test, y_test = X_test.to(device), y_test.to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Using Adam optimizer

    cost = []

    for epoch in range(num_epochs):
        if shuffle:
            idx = torch.randperm(y.size(0))
            x, y = x[idx], y[idx]

        optimizer.zero_grad()  
        yhat = model(x)
        loss_value = criterion(yhat, y)
        loss_value.backward()  
        optimizer.step()  

        if X_test is not None and y_test is not None:
            with torch.no_grad():
                test_loss = criterion(model(X_test), y_test).item()
            print(f"Epoch {epoch+1:03d} | MSE: {test_loss:.5f}")
            cost.append(test_loss)
    
    return cost

In [35]:
torch.manual_seed(123)

models = []

for i in range(1):
    shuffle_idx = torch.randperm(X.size(0), dtype=torch.long)

    X, y = X[shuffle_idx], Y[i][shuffle_idx]


    percent70 = int(shuffle_idx.size(0)*0.7)

    X_train, X_test = X[shuffle_idx[:percent70]], X[shuffle_idx[percent70:]]
    y_train, y_test = y[shuffle_idx[:percent70]], y[shuffle_idx[percent70:]]

    # Normalize (mean zero, unit variance)
    mu, sigma = X_train.mean(dim=0), X_train.std(dim=0)
    X_train = (X_train - mu) / sigma
    X_test = (X_test - mu) / sigma

    models.append(LinearRegression(input_dim=X_train.size(1)))
    cost1 = train(models[i], 
                X_train, y_train, 
                num_epochs=1000, 
                learning_rate=0.005,
                shuffle=True,
                X_test=X_test,
                y_test=y_test
                )

    train_pred = models[i].forward(X_train)
    test_pred = models[i].forward(X_test)

    print('Model %03d' % i)
    print('Train MSE: %.5f' % loss(train_pred, y_train))
    print('Test MSE: %.5f' % loss(test_pred, y_test))

Epoch 001 | MSE: 16826.75195
Epoch 002 | MSE: 16754.36719
Epoch 003 | MSE: 16655.60742
Epoch 004 | MSE: 16524.72852
Epoch 005 | MSE: 16355.44629
Epoch 006 | MSE: 16140.81152
Epoch 007 | MSE: 15873.86426
Epoch 008 | MSE: 15547.42773
Epoch 009 | MSE: 15154.11621
Epoch 010 | MSE: 14686.59473
Epoch 011 | MSE: 14138.10352
Epoch 012 | MSE: 13503.15234
Epoch 013 | MSE: 12778.12500
Epoch 014 | MSE: 11961.64844
Epoch 015 | MSE: 11055.71094
Epoch 016 | MSE: 10066.68359
Epoch 017 | MSE: 9007.58691
Epoch 018 | MSE: 7899.12793
Epoch 019 | MSE: 6772.11719
Epoch 020 | MSE: 5669.61133
Epoch 021 | MSE: 4646.99414
Epoch 022 | MSE: 3768.22632
Epoch 023 | MSE: 3090.62598
Epoch 024 | MSE: 2639.19165
Epoch 025 | MSE: 2392.13745
Epoch 026 | MSE: 2303.73901
Epoch 027 | MSE: 2336.30591
Epoch 028 | MSE: 2468.38208
Epoch 029 | MSE: 2687.27856
Epoch 030 | MSE: 2979.23535
Epoch 031 | MSE: 3324.94995
Epoch 032 | MSE: 3699.54834
Epoch 033 | MSE: 4075.40039
Epoch 034 | MSE: 4426.60742
Epoch 035 | MSE: 4731.88135
Epoc

In [157]:
# Open the image
image = Image.open('output.png').convert('L')

# Prepare the image
features = np.array(image).flatten()
features = torch.tensor([features], dtype=torch.float)
mu, sigma = X.mean(dim=0), X.std(dim=0)
features = (features - mu) / sigma

# Predict
output_data = []
for i in range(500):
    output = models[i].forward(features)
    output_data.append(output)

image = Image.new('L', (100, 5))

# Put the pixels into the image
image.putdata(output_data)

# Save the image
image.save('output2.png')

